In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
#Saving paths that'll be called on
results_path='../Results/'

In [3]:
#Reading in results from different models
InceptionV3=pd.read_csv(results_path+'InceptionV3/InceptionV3 Regression Optimization.csv')
MobileNet=pd.read_csv(results_path+'MobileNet/MobileNet Regression Optimization.csv')
ResNet50=pd.read_csv(results_path+'ResNet50/ResNet50 Regression Optimization.csv')
VGG19=pd.read_csv(results_path+'VGG19/VGG19 Regression Optimization.csv')
Xception=pd.read_csv(results_path+'Xception/Xception Regression Optimization.csv')

In [4]:
#Combining results together
df=pd.concat([InceptionV3,MobileNet,ResNet50,VGG19,Xception], axis=0).reset_index(drop=True)

In [5]:
#Check
df.head()

,model,new layers before flatten,nodes,dropout,lr,loss function,train_loss,other metrics,test_MSE,test_R2,epochs,early_stopping,stop_patience
0,InceptionV3,4,"[27, 27]","[0.5, 0.5]",0.000075,mean_squared_error,632.012,"[{'mae': 19.521556854248047}, {'mape': 83.1570...",888.409961,-0.326439,150,NaN,NaN
1,MobileNet,5,"[28, 5]","[0.3, 0.3]",0.000080,mean_squared_error,588.269,"[{'mae': 18.6601619720459}, {'mape': 88.542747...",814.492553,-0.216077,100,F,50.0
2,MobileNet,7,"[350, 300, 75, 15]","[0.5, 0.5, 0.5, 0]",0.000080,mean_squared_error,179.754,"[{'mae': 10.53239917755127}, {'mape': 42.76624...",1072.141784,-0.600760,150,"T, epoch 93",50.0
3,MobileNet,8,"[200, 100, 50, 28]","[0.5, 0.5, 0.5, 0.5]",0.000050,mean_squared_error,675.705,"[{'mae': 20.29682731628418}, {'mape': 65.31738...",962.894692,-0.437649,150,F,50.0
4,MobileNet,6,"[30, 30, 30]","[0.5, 0.5, 0.5]",0.000050,mean_squared_error,913.825,"[{'mae': 24.04484748840332}, {'mape': 90.32450...",966.038502,-0.442343,150,F,50.0


In [6]:
#Find the best test MSE score from each model by grouping by model, then taking the min test MSE
result_df=df[df['test_MSE'].isin(df.groupby('model')['test_MSE'].min().values)]
result_df.reset_index(drop=True, inplace=True)

In [7]:
#Round results up to 2 decimal points and sort test MSE value
result_df['test_MSE']=round(result_df['test_MSE'],2)
result_df['test_R2']=round(result_df['test_R2'],2)
result_df=result_df.sort_values(['test_MSE']).reset_index(drop=True)

C:\Users\User\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#Look at the metrics used to evaluate test performance
result_df[['model', 'test_MSE', 'test_R2']]

,model,test_MSE,test_R2
0,MobileNet,795.87,-0.19
1,Xception,843.20,-0.26
2,InceptionV3,888.41,-0.33
3,ResNet50,1006.39,-0.50
4,VGG19,1010.03,-0.51


In [9]:
#Get the parameter settings for the best run of each model

#Create an empty column
result_df['title']=''

#Loop through each row
for i in range(0, len(result_df)):
    
    #Get the parameter settings from each column and save it to variables
    model=result_df.loc[i, 'model']
    #Since results are returned as strings, the number of layers will need to be split by ','
    #to become a list for len() to get the correct number of layers
    dense_layers=len(result_df.loc[i, 'nodes'].replace("[", "").replace("]","").replace(" ","").split(','))
    nodes=result_df.loc[i, 'nodes']
    dropout=result_df.loc[i, 'dropout']
    lr=result_df.loc[i, 'lr']
    loss_function=result_df.loc[i, 'loss function']
    epochs=result_df.loc[i, 'epochs']
    early_stop=result_df.loc[i, 'early_stopping']
    pat=result_df.loc[i, 'stop_patience']
    
    result_df.loc[i, 'title']='Model: {}, {} Dense layers, Nodes: {}, Dropout: {}, Learning rate: {}, Loss function: {}, \
Epochs: {}, Early stop: {}, Stop patience: {}\
'.format(model, dense_layers, nodes, dropout, lr, loss_function, epochs, early_stop, pat)

In [10]:
#Get the best performing model and its parameter settings
result_df['title'][0]

'Model: MobileNet, 2 Dense layers, Nodes: [27, 27], Dropout: [0.5, 0.5], Learning rate: 7.5e-05, Loss function: mean_squared_error, Epochs: 150, Early stop: nan, Stop patience: nan'